# App for Dataviz

In [1]:

import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


## import data

In [2]:
# read dataframe (preprocessed in main.ipynb)
df = pd.read_csv("data/merged_data.csv")
df.head()

,Unnamed: 0,average_life_expectancy,population,year,Country,Alpha-2 code,Alpha-3 code,id,Latitude (average),Longitude (average)
0,0,37.409,11174000,1970,Afghanistan,AF,AFG,4,33.0,65.0
1,0,37.930,11475000,1971,Afghanistan,AF,AFG,4,33.0,65.0
2,0,38.461,11791000,1972,Afghanistan,AF,AFG,4,33.0,65.0
3,0,39.003,12109000,1973,Afghanistan,AF,AFG,4,33.0,65.0
4,0,39.558,12413000,1974,Afghanistan,AF,AFG,4,33.0,65.0


## Modify Data

In [3]:
# codes needed to shrink down the dataset to the african countries we want to display
african_iso_codes = ["DZA"
,"AGO"
,"BEN"
,"BWA"
,"BFA"
,"BDI"
,"CMR"
,"CPV"
,"CAF"
,"TCD"
,"COM"
,"COG"
,"COD"
,"CIV"
,"DJI"
,"EGY"
,"GNQ"
,"ERI"
,"ETH"
,"GAB"
,"GMB"
,"GHA"
,"GIN"
,"GNB"
,"KEN"
,"LSO"
,"LBR"
,"LBY"
,"MDG"
,"MLI"
,"MWI"
,"MRT"
,"MUS"
,"MYT"
,"MAR"
,"MOZ"
,"NAM"
,"NER"
,"NGA"
,"REU"
,"RWA"
,"STP"
,"SEN"
,"SYC"
,"SLE"
,"SOM"
,"ZAF"
,"SSD"
,"SDN"
,"SWZ"
,"TZA"
,"TGO"
,"TUN"
,"UGA"
,"ESH"
,"ZMB"
,"ZWE"]
# source: https://www.nro.net/list-of-country-codes-in-the-afrinic-region/

In [4]:
# shrink dataset to african countries
df = df[df["Alpha-3 code"].isin(african_iso_codes)]

In [5]:
df.head()

,Unnamed: 0,average_life_expectancy,population,year,Country,Alpha-2 code,Alpha-3 code,id,Latitude (average),Longitude (average)
102,2,50.372,14465000,1970,Algeria,DZ,DZA,12,28.0,3.0
103,2,50.773,14872000,1971,Algeria,DZ,DZA,12,28.0,3.0
104,2,51.201,15286000,1972,Algeria,DZ,DZA,12,28.0,3.0
105,2,51.676,15710000,1973,Algeria,DZ,DZA,12,28.0,3.0
106,2,52.218,16149000,1974,Algeria,DZ,DZA,12,28.0,3.0


## Load Geojson data

In [6]:
# source: https://public.opendatasoft.com/explore/dataset/country_shapes/export/ 
import geojson
with open("data/country_shapes.geojson") as f:
    gj = geojson.load(f)
gj.get

<function FeatureCollection.get(key, default=None, /)>

## Create App

In [7]:
# the variable that holds our final webAPP
app = JupyterDash(__name__)

# here we define the layout and components of our app
app.layout = html.Div([
    #  africa map container
    html.Div([
        html.P("Life expectancy:"), # heading
        dcc.Graph(id="choropleth"), # africa map figure with id "chloropleth"
    ], style={'width': '100%', 'height':'90vh', 'margin': '0px 20px 20px 20px'}),
    # slider for year selection
    html.Div(dcc.Slider(
            id='slider',
            min=df['year'].min(),
            max=df['year'].max(),
            value=df['year'].max(),
            marks={str(year): str(year) for year in df['year'].unique()},
            step=None
        ), style={'width': '100%', 'margin': '0px 20px 20px 20px'}),
    # dropdown for country selection
    html.Div(dcc.Dropdown(
                    id='country_select',
                    options=[{'label': i, 'value': i} for i in african_iso_codes],
                    multi=True
                )),
    # graph (later for the wormgraph)
    html.Div([dcc.Graph(id="worm_figure")], style={'display':'block','margin': '20px 20px 20px 20px'}, id="worm_container")
            ])

## Create Callback

In [8]:
# here we define the functionality and interactivity of our App
# define input and output of callback function
@app.callback(
    Output("choropleth", "figure"),
    Output("worm_figure", "figure"),
    # Output("worm_container", "style"),
    Input("slider", "value"),
    Input("country_select", "value"))
# the callback function: called when a value of the defined inputs above changes.
def display_choropleth(slider, country_select):
    # build the africa map
    fig = px.choropleth(
        df[df.year == slider], geojson=gj, color="average_life_expectancy",
        locations="Alpha-3 code", featureidkey="properties.cou_iso3_code",
        projection="orthographic", range_color=[0, 90])
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, autosize = True)

    # worm_style = {'display', 'block'}
    # if any countries are selected
    if(country_select != None):
        fig2 = px.line(df[df["Alpha-3 code"].isin(country_select)], x="year", y="average_life_expectancy")
        fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, autosize = True)
        worm_style = {'display', 'block'}
    # else (should actually hide the bottom graph, but that did not work yet)
    else:
        fig2 = px.line(df,  x="year", y="average_life_expectancy")
    # return (what we have defined above)
    return fig , fig2 #, worm_style

## RUN APP Server

In [9]:
# run the local server
# visit http://127.0.0.1:8050/ in webbrowser to see results and error codes
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
